In [1]:
import os
import sys
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci
import traci.constants as tc
import random

In [3]:
# Random or locked seed, depending if needing data or reproducability
rand_seed = random.randint(1,100000)
#rand_seed = 50000
print(rand_seed)

# Random Start Time
start_time = random.randint(1,2400)
print(start_time)

# Random Duration
duration = random.randint(1150,1250)

# Start time 2 randomly assigned
# Comment out if hack duration 15 minutes or over, to allow time for the hacks to reset inside of the 1 hour deadline
#start_2 = start_time+duration+random.randint(50,800)

# Start time 2 eliminator (used for long hack durations 15+ when only running an hour)
start_2 = -1

print(start_2)
print(duration)

# Insert own sumo file path, this is for my personal PC
sumoBinary = # "Path to SUMO Gui in local file system"
sumoCmd = [sumoBinary, "-c", '''#Path to sumoCFG file in local file system''',"--seed", str(rand_seed)]

61855
1809
-1
1231
['/Program Files (x86)/Eclipse/Sumo/bin/sumo-gui.exe', '-c', 'C:\\Users\\jacqu\\Sumo\\2024-03-20-19-50-20\\daytona_smaller.sumocfg', '--seed', '61855']


In [4]:
import traci
# Port and setOrder used if having multiple VMs connecting and controlling individual lights, comment out if only running locally
#PORT = 65431
traci.start(sumoCmd)
#traci.setOrder(1) # number can be anything

step = 0

# Variable to determine which attack is used and which light is affected
hack = 0
light = 0



# Data Sets for TraCI

max_jam_length_m = []
interval_mean_speed = []
interval_occupancy = []
interval_vehicle_number = []


# Reproducing the normal traffic light phase pattern to add it back after changing it for light 1
phases_l1 = []

# Actuated Method: Light 1

phases_l1.append(traci.trafficlight.Phase(29, "GGGGrrrrrrrGGGGrrrrrrr", minDur="9", maxDur="50", next=()))
phases_l1.append(traci.trafficlight.Phase(5, "yyyyrrrrrrryyyyrrrrrrr", minDur="5", maxDur="5", next=()))
phases_l1.append(traci.trafficlight.Phase(20, "rrrrGGrrrrrrrrrGGrrrrr", minDur="9", maxDur="50", next=()))
phases_l1.append(traci.trafficlight.Phase(5, "rrrryyrrrrrrrrryyrrrrr",  minDur="5", maxDur="5", next=()))
phases_l1.append(traci.trafficlight.Phase(29, "rrrrrrGGGggrrrrrrGGGgg", minDur="7", maxDur="50", next=()))
phases_l1.append(traci.trafficlight.Phase(5, "rrrrrryyyggrrrrrryyygg", minDur="5", maxDur="5", next=()))
phases_l1.append(traci.trafficlight.Phase(20, "rrrrrrrrrGGrrrrrrrrrGG", minDur="7", maxDur="50", next=()))
phases_l1.append(traci.trafficlight.Phase(5, "rrrrrrrrryyrrrrrrrrryy", minDur="5", maxDur="5", next=()))


logic_1 = traci.trafficlight.Logic("my_program", type=tc.TRAFFICLIGHT_TYPE_ACTUATED, currentPhaseIndex=0, phases=phases_l1)

# Reproducing the normal traffic light phase pattern to add it back after changing it for light 2
phases_l2 = []

# Actuated Method: Light 2

phases_l2.append(traci.trafficlight.Phase(41, "rrrrGGGgrrrrGGGgg", minDur="7", maxDur="50", next=()))
phases_l2.append(traci.trafficlight.Phase(4, "rrrryyyyrrrryyyyy", minDur="4", maxDur="4", next=()))
phases_l2.append(traci.trafficlight.Phase(41, "GGggrrrrGGggrrrrr", minDur="5", maxDur="50", next=()))
phases_l2.append(traci.trafficlight.Phase(4, "yyyyrrrryyyyrrrrr",  minDur="4", maxDur="4", next=()))


logic_2 = traci.trafficlight.Logic("my_program", type=tc.TRAFFICLIGHT_TYPE_ACTUATED, currentPhaseIndex=0, phases=phases_l2)

# Light 1 Section
#  Hack 0 : All lights red to collect cars, all green to cause the optimal amount of confusion or collisions
if light == 1:
    if hack == 0:
        step_2 = -1
        final_step = -1
        step_2_2 = -1
        final_step_2 = -1
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:
            if step == start_time:
                traci.trafficlight.setRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741','rrrrrrrrrrrrrrrrrrrrrrr')
                step_2 = round(start_time+duration/2)
            if step == step_2:
                traci.trafficlight.setRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741','GGGGGGGGGGGGGGGGGGGGGGG')
                final_step = round(step_2+duration/2)
            if step == final_step:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            if step == start_2:
                traci.trafficlight.setRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741','rrrrrrrrrrrrrrrrrrrrrrr')
                step_2_2 = round(start_2+duration/2)
            if step == step_2_2:
                traci.trafficlight.setRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741','GGGGGGGGGGGGGGGGGGGGGGG')
                final_step_2 = round(step_2_2+duration/2)
            if step == final_step_2:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)

            traci.simulationStep()
            step += 1

    # No hack for testing initial pattern
    elif hack == 999:
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:
            traci.simulationStep()
            # West
            vehicles_at_junction_1 = traci.edge.getLastStepVehicleIDs('880913510')
            # South
            vehicles_at_junction_2 = traci.edge.getLastStepVehicleIDs('845617904')
            # East
            vehicles_at_junction_3 = traci.edge.getLastStepVehicleIDs('1074432362#0')
            # North
            vehicles_at_junction_4 = traci.edge.getLastStepVehicleIDs('880072670')
            step += 1

    # Hack 1: 5 Second Traffic Delay Theory
    elif hack ==1:
        end_time = -1
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:
            print(traci.trafficlight.getPhaseName('cluster_101082813_749801512_7889380740_7889380741'))
            if step == start_time:
                traci.trafficlight.setRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741','rrrrrrrrrrrrrrrrrrrrrrr')
                end_time = start_time+5
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            traci.simulationStep()
            step += 1

    # Making more attacks that are IRL-realistic due to MMU's

    # Hack 2: Lots of Yellow Lights (switching time of yellow and green lights)
    elif hack ==2:
        end_time = -1
        end_time_2 = -1
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:        
            print(traci.trafficlight.getPhaseName('cluster_101082813_749801512_7889380740_7889380741'))
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(5, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(5, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_time_2 = start_2+duration
            if step == end_time_2:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            traci.simulationStep()
            #print(traci.trafficlight.getRedYellowGreenState('cluster_101082813_749801512_7889380740_7889380741'))
            step += 1

    # Hack 3: Extremely short yellow lights. Could cause collisions with people thinking they can make it in time
    # Note: Doesn't do much in simulation potentially due to how software deals with collisions
    
    elif hack == 3:
        end_time = -1
        end_time_2 = -1
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(29, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(1, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(29, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(1, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(20, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_time_2 = start_2+duration
            if step == end_time_2:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            traci.simulationStep()
            step += 1


    # Hack 4: Bias towards going straight, causing a theoretical traffic jam for people both wanting to turn to either side
    elif hack ==4:
        end_time = -1
        end_2 = -1
        traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
        while step <= 3600:
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(50, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(50, "GGGGrrrrrrrGGGGrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "yyyyrrrrrrryyyyrrrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGrrrrrrrrrGGrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrryyrrrrrrrrryyrrrrr", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrGGGggrrrrrrGGGgg",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrryyyggrrrrrryyygg", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrrGGrrrrrrrrrGG",next=()))
                phases_2.append(traci.trafficlight.Phase(5, "rrrrrrrrryyrrrrrrrrryy",next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', hacked_logic_1)
                end_2 = start_2+duration
            if step == end_2:
                traci.trafficlight.setProgramLogic('cluster_101082813_749801512_7889380740_7889380741', logic_1)
            traci.simulationStep()
            step += 1
            
# Light 2 Section

elif light == 2:
    # Hack 0
    if hack == 0:
        step_2 = -1
        final_step = -1
        step_2_2 = -1
        final_step_2 = -1
        while step <= 3600:

            if step == start_time:
                traci.trafficlight.setRedYellowGreenState('cluster_1715694384_3655880346_3657137041','rrrrrrrrrrrrrrrrr')
                step_2 = round(start_time+duration/2)
                print(step_2)
            if step == step_2:
                traci.trafficlight.setRedYellowGreenState('cluster_1715694384_3655880346_3657137041','GGGGGGGGGGGGGGGGG')
                final_step = round(step_2+duration/2)
            if step == final_step:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
                print('Did')
            if step == start_2:
                traci.trafficlight.setRedYellowGreenState('cluster_1715694384_3655880346_3657137041','rrrrrrrrrrrrrrrrr')
                step_2_2 = round(start_2+duration/2)
            if step == step_2_2:
                traci.trafficlight.setRedYellowGreenState('cluster_1715694384_3655880346_3657137041','GGGGGGGGGGGGGGGGG')
                final_step_2 = round(step_2_2+duration/2)
            if step == final_step_2:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
                print('Did')

            traci.simulationStep()
            step += 1

    # No hack for testing initial pattern
    elif hack == 999:
        while step <= 3600:
            traci.simulationStep()
            # West
            vehicles_at_junction_1 = traci.edge.getLastStepVehicleIDs('880913510')
            # South
            vehicles_at_junction_2 = traci.edge.getLastStepVehicleIDs('845617904')
            # East
            vehicles_at_junction_3 = traci.edge.getLastStepVehicleIDs('1074432362#0')
            # North
            vehicles_at_junction_4 = traci.edge.getLastStepVehicleIDs('880072670')
            step += 1

    # Hack 1: 5 Second Traffic Delay Theory
    elif hack ==1:
        end_time = -1
        while step <= 3600:
            print(traci.trafficlight.getPhaseName('cluster_1715694384_3655880346_3657137041'))
            if step == start_time:
                traci.trafficlight.setRedYellowGreenState('cluster_1715694384_3655880346_3657137041','rrrrrrrrrrrrrrrrr')
                end_time = start_time+5
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            traci.simulationStep()
            step += 1
    # Making more attacks that are IRL-realistic due to MMU's

    # Hack 2: Lots of Yellow Lights
    elif hack ==2:
        end_time = -1
        end_time_2 = -1
        while step <= 3600:        
            print(traci.trafficlight.getPhaseName('cluster_1715694384_3655880346_3657137041'))
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(5, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(20, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_time_2 = start_2+duration
            if step == end_time_2:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            traci.simulationStep()
            step += 1

    # Hack 3: Extremely short yellow lights. Could cause collisions with people thinking they can make it
    elif hack == 3:
        end_time = -1
        end_time_2 = -1
        while step <= 3600:
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(29, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(29, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(1, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(29, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(1, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_time_2 = start_2+duration
            if step == end_time_2:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            traci.simulationStep()
            step += 1


    # Hack 4: Bias towards going straight, causing a theoretical traffic jam for people both wanting to turn left and across the other way
    elif hack ==4:
        end_time = -1
        end_2 = -1
        while step <= 3600:
            if step == start_time:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(50, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(3, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(3, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_time = start_time+duration
            if step == end_time:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            if step == start_2:
                phases_2 = []
                phases_2.append(traci.trafficlight.Phase(50, "rrrrGGGgrrrrGGGgg", next=()))
                phases_2.append(traci.trafficlight.Phase(3, "rrrryyyyrrrryyyyy", next=()))
                phases_2.append(traci.trafficlight.Phase(5, "GGggrrrrGGggrrrrr",next=()))
                phases_2.append(traci.trafficlight.Phase(3, "yyyyrrrryyyyrrrrr", next=()))
                hacked_logic_1 = traci.trafficlight.Logic("hacked-program-1", 0, 0, phases=phases_2)
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', hacked_logic_1)
                end_2 = start_2+duration
            if step == end_2:
                traci.trafficlight.setProgramLogic('cluster_1715694384_3655880346_3657137041', logic_2)
            traci.simulationStep()
            step += 1


traci.close()


# 